# Avisos

Este notebook contem os scrips que irão:

- Coletar os dados nas tabelas da Stage;
- Criar as queries de tratamentos e cargas dos dados;
- Carregar esses dados tratados nas tabelas de Dimensão definidas no DW.

# Imports

In [167]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

# Variáveis

In [168]:
# Variável que determina a data da carga de uma tabela.
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')

# Variável com o diretório local onde serão salvos os bancos de dados SQLite.
endereco = Path('G:\\Meu Drive\\Projetos - Data Engineer\\Databases\\')

# Definindo o local de criação dos bancos.
BDODS = endereco / "ibgeSTG.db"                  #ODS = Operational Data Store
BDDW = endereco / "ibgeDW.db"                    #DW = Data Warehouse

# Conectando nos bancos de dados.
connODS = sql.connect(BDODS)
connDW = sql.connect(BDDW)

# Definindo a manipulação de dados via SQL.
SQL_ODS = connODS.cursor()
SQL_DW = connDW.cursor()

# Criando os dataframes com as queries que irão carregar os dados

In [169]:
# Definindo a query de carga na dim_municipio.

qry_dim_municipio = '''
SELECT
	cod_municipio,
	nm_municipio
FROM
(
        SELECT
            a.cod_municipio,
            a.nm_municipio,
            MAX(A.dtCarga) AS ultima_Carga
        FROM st_municipio a
            WHERE a.dtCarga = (
                                SELECT
                                MAX(x.dtCarga)
                                FROM st_municipio x
                                )
        GROUP BY 
            a.cod_municipio,
            a.nm_municipio
) A
'''

# Criando o dataframe.
df_dim_municipio = pd.read_sql(qry_dim_municipio,connODS)

df_dim_municipio

,cod_municipio,nm_municipio
0,3300100,Angra dos Reis
1,3300159,Aperibé
2,3300209,Araruama
3,3300225,Areal
4,3300233,Armação dos Búzios
...,...,...
87,3306008,Três Rios
88,3306107,Valença
89,3306156,Varre-Sai
90,3306206,Vassouras


In [170]:
# Definindo a query de carga na dim_dp.

qry_dim_dp = '''
SELECT
	cod_dp,
	nm_dp,
	endereco,
	nm_responsavel
FROM
(
		SELECT
			a.cod_dp,
			a.nm_dp,
			a.endereco,
			b.nm_responsavel,
			MAX(A.dtCarga) AS ultima_Carga
		FROM st_dp a
		JOIN st_resp_dp b
			ON a.cod_dp = b.cod_dp
            WHERE a.dtCarga = (
                                SELECT
                                MAX(x.dtCarga)
                                FROM st_dp x
                                )
		GROUP BY 
			a.cod_dp,
			a.nm_dp,
			a.endereco,
			b.nm_responsavel
) A
'''

# Criando o dataframe.
df_dim_dp = pd.read_sql(qry_dim_dp,connODS)

In [171]:
# Definindo a query de carga na dim_bpm.

qry_dim_bpm = '''
SELECT 
    cod_bpm,
    nm_bpm,
    endereco,
    area_bpm
FROM (
        SELECT
        a.cod_bpm,
        a.nm_bpm,
        a.endereco,
        b.area_bpm,
        MAX(a.dtCarga)
        FROM st_bpm a
        JOIN st_area_bpm b
            ON a.cod_bpm = b.cod_bpm
        WHERE a.dtCarga = (
                            SELECT
                            MAX(x.dtCarga)
                            FROM st_bpm x
        )
GROUP BY
    a.cod_bpm,
    a.nm_bpm,
    a.endereco,
    b.area_bpm
) A
'''


# Criando o dataframe.
df_dim_bpm = pd.read_sql(qry_dim_bpm,connODS)

df_dim_bpm

,cod_bpm,nm_bpm,endereco,area_bpm
0,2,2º BPM,"R. Álvaro Ramos, 155",15.0
1,3,3º BPM,"R. Lucídio Lago, 181",39.0
2,4,4º BPM,"R. Francisco Eugênio, 228",17.1
3,5,5º BPM,"Praça Cel. Assunção, S/N",15.4
4,6,6º BPM,"R. Barão de Mesquita, 625",55.2
5,7,7º BPM,"R. Dr. Alfredo Backer, 367",247.7
6,8,8º BPM,"Rua Tenente Coronel Cardoso, s/n",6635.7
7,9,9º BPM,"R. Tacaratu, 94",33.6
8,10,10º BPM,"Rod. Lúcio Meira, 47000",4243.6
9,11,11º BPM,"R. Voluntários da Pátria, 474",4041.1


# Carregando os dados na dim_tempo

In [172]:
# Criando a CTE que será utilizada para carregar os dados na dim_tempo.

qry_dim_tempo = """
-- CTE que retorna a data inicial da minha tabela de tempo.

WITH data(dataini) AS
    (
       VALUES('2018-01-01')             -- Setando apenas uma data referência, com base na data inicial da tabela Fato.
       UNION ALL
       SELECT date(dataini, '+1 month') -- Definindo o incremento mensal, pois essa é a granularidade da tabela Fato.
       FROM data
       WHERE dataini < '2021-12-01'     -- Para situações onde há cargas diárias, pode ser utilizada a função: DATE('now')
   )
SELECT
    CAST(STRFTIME('%Y', dataini) AS TEXT) || CAST(STRFTIME('%m', dataini) AS TEXT) AS sk_tempo,
    STRFTIME('%d/%m/%Y', dataini) AS data,
    CAST(STRFTIME('%d', dataini) AS INTEGER) AS dia,
    CAST(STRFTIME('%m', dataini) AS INTEGER) AS mes,
    CAST(STRFTIME('%Y', dataini) AS INTEGER) AS ano,
    CASE
        WHEN CAST(STRFTIME('%m', dataini) AS INTEGER) IN (1,2,3) THEN 'T1'
        WHEN CAST(STRFTIME('%m', dataini) AS INTEGER) IN (4,5,6) THEN 'T2'
        WHEN CAST(STRFTIME('%m', dataini) AS INTEGER) IN (7,8,9) THEN 'T3'
                                                                 ELSE 'T4'
    END AS trimestre,
        CASE
        WHEN CAST(STRFTIME('%m', dataini) AS INTEGER) IN (1,2,3,4,5,6) THEN 'S1'
                                                                       ELSE 'S2'
    END AS semestre
FROM data
"""

# Carregando dados no dataframe.
df_dim_tempo = pd.read_sql(qry_dim_tempo, connDW)

In [173]:
# Deletando os dados antigos na tabela dim_tempo.
SQL_DW.execute("DELETE FROM dim_tempo")

# Reiniciando o autoincramental da chave.
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dim_tempo'")

# Confirmando a transação.
connDW.commit()

In [174]:
# Inserindo registros na tabela dim_tempo.

print('Iniciando a carga de dados na tabela dim_tempo')
print('--------------------------------------------')


connDW.executemany('''INSERT INTO dim_tempo(sk_tempo, data, dia, mes, ano, trimestre, semestre) VALUES(?,?,?,?,?,?,?)''', df_dim_tempo.values.tolist())
connDW.commit()

# Exibindo mensagem de conclusão da carga.
print('Carga finalizada!', len(df_dim_tempo), 'registros inseridos na dim_municipio.')
print('--------------------------------------------')

Iniciando a carga de dados na tabela dim_tempo
--------------------------------------------
Carga finalizada! 48 registros inseridos na dim_municipio.
--------------------------------------------


# Executando a carga das Dimensões no DW

In [175]:
# Deletando os dados antigos na tabela dim_municipio.
SQL_DW.execute("DELETE FROM dim_municipio")

# Reiniciando o autoincramental da chave.
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dim_municipio'")

# Confirmando a transação.
connDW.commit()

In [176]:
# Inserindo registros na tabela dim_municipio.

print('Iniciando a carga de dados na tabela dim_municipio')
print('--------------------------------------------')


connDW.executemany('''INSERT INTO dim_municipio(cod_municipio, nm_municipio) VALUES(?,?)''', df_dim_municipio.values.tolist())
connDW.commit()

# Exibindo mensagem de conclusão da carga.
print('Carga finalizada!', len(df_dim_municipio), 'registros inseridos na dim_municipio.')
print('--------------------------------------------')

Iniciando a carga de dados na tabela dim_municipio
--------------------------------------------
Carga finalizada! 92 registros inseridos na dim_municipio.
--------------------------------------------


In [177]:
# Deletando os dados antigos na tabela dim_dp.
SQL_DW.execute("DELETE FROM dim_dp")

# Reiniciando o autoincramental da chave.
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dim_dp'")

# Confirmando a transação.
connDW.commit()

In [178]:
# Inserindo registros na tabela dim_dp.

print('Iniciando a carga de dados na tabela dim_dp')
print('--------------------------------------------')


connDW.executemany('''INSERT INTO dim_dp(cod_dp, nm_dp, endereco, nm_responsavel) VALUES(?,?,?,?)''', df_dim_dp.values.tolist())
connDW.commit()

# Exibindo mensagem de conclusão da carga.
print('Carga finalizada!', len(df_dim_dp), 'registros inseridos na dim_dp.')
print('--------------------------------------------')

Iniciando a carga de dados na tabela dim_dp
--------------------------------------------
Carga finalizada! 137 registros inseridos na dim_dp.
--------------------------------------------


In [179]:
# Deletando os dados antigos na tabela dim_bpm.
SQL_DW.execute("DELETE FROM dim_bpm")

# Reiniciando o autoincramental da chave.
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dim_bpm'")

# Confirmando a transação.
connDW.commit()

In [180]:
# Inserindo registros na tabela dim_bpm.

print('Iniciando a carga de dados na tabela dim_bpm')
print('--------------------------------------------')


connDW.executemany('''INSERT INTO dim_bpm(cod_bpm, nm_bpm, endereco, area_bpm) VALUES(?,?,?,?)''', df_dim_bpm.values.tolist())
connDW.commit()

# Exibindo mensagem de conclusão da carga.
print('Carga finalizada!', len(df_dim_bpm), 'registros inseridos na dim_bpm.')
print('--------------------------------------------')

Iniciando a carga de dados na tabela dim_bpm
--------------------------------------------
Carga finalizada! 39 registros inseridos na dim_bpm.
--------------------------------------------


# Fechando as conexões

In [181]:
# Fechando as conexões com ODS e DW.

connODS.close()
connDW.close()